# Steam TOP 250 Game Web-Scraping

In this kernel, I will attempt to web-scrape video games in steam platform,which is a world-class video game digital distribution service by Valve.

### 1. Import Libraries
we first import the necessary modules:
   1. request
   2. pandas
   3. re
   4. BeautifulSoup from bs4
   5. Thread from threading
   6. sleep from time

In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

### 2. Create self-definition function get_text(url)
next, we need to retrieve the text information from the URL by using request module. In order to avoid unnecessary error information interrupt our process, we use 'try & exception' to continue our coding. 

In [2]:
def get_text(url):
    try:
        headers = {
            "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/85.0.4183.102 Safari/537.36', 'Accept-Language': 'en-US '
        }
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "Webscraping Fails！"

### 3. initialize lists for dataframe creation
Then we create empty dictionary and lists for dataframe which we will use for analysis afterwards. In this section, we focus on several factors shown below:
   1. Game Name
   2. Game Release Date
   3. Game Genre
   4. Game Price
   5. Score
   6. Rating
   7. Votes

In [3]:
game_info = dict()
num = 1
name = []
date = []
genre = []
price = []
score = []
rating = []
votes = []

### 3. confirm the link
After that, we need to confirm the link of our data collection: https://steam250.com/top250.

In [4]:
link = "https://steam250.com/top250"

### 4. create self-definition function to get text information of target URL for further use
When we initialize the empty lists and dictionary for further use, we need to use BeautifulSoup function from bs4 module to get text info of the link.

In [5]:
def get_name_text():
    text = get_text(link)
    soup = BeautifulSoup(text, "html.parser")
    text_info = soup.find_all('div', id = num)
    return text_info

### 5. create self-definition function to append information from web-scraping method into related list
After we get the text information of the link, we need to use find & find_all functions to append information into our lists.

In [6]:
def enrich_name(): 
    for a in text_info:
        b = a.find_all('span', class_= 'title')
        for x in b:
            name.append(x.find('a').text)
    return name

In [7]:
def enrich_date(): 
    for a in text_info:
        if a.find_all('span', class_= 'date') == []:
            b = None
        else:
            b = a.find_all('span', class_= 'date')
        if b != None:
             for c in b:
                date.append(c.find('a').text[1 : -1].replace(' ', '-'))
        else:
            date.append(b)
    return date

In [8]:
def enrich_genre(): 
    for a in text_info:
        b = a.find_all('a', class_= 'genre')
        for x in b:
            genre.append(x.text)
    return genre

In [9]:
def enrich_price(): 
    for a in text_info:
        if a.find_all('a', class_= 'free') == []:
            b = a.find_all('span', class_= 'price')
        else:
            b = '$0.00' 
        if b != '$0.00':
             for c in b:
                price.append(c.text[1: ])
        else:
            price.append(b[1: ])
    return price

In [10]:
def enrich_score(): 
    for a in text_info:
        b = a.find_all('span', class_= 'score')
        for x in b:
            score.append(x.text)
    return score
    sleep(0.5)

In [11]:
def enrich_rating(): 
    for a in text_info:
        b = a.find_all('span', class_= 'rating')
        for x in b:
            if x.text == '100%':
                rating.append('1')
            else:    
                rating.append('0.' + x.text[ : -1])
    return rating

In [12]:
def enrich_votes(): 
    for a in text_info:
        b = a.find_all('span', class_= 'votes')
        for x in b:
            votes.append(x.text[ : -6].replace(',', ''))
    return votes

### 6. Iterate and append info into our list 
since we have 250 records about related games, we use iteration to repeat the methods or functions that we create before to continuously append information into our lists until the end.

In [13]:
while num <= 250:
    text_info = get_name_text()
    enrich_name()
    enrich_date()
    enrich_genre()
    enrich_price()
    enrich_score()
    enrich_rating()
    enrich_votes()
    num += 1

### 7. Append the information of the lists into our dictionary.
Remember the empty dictionary that we create before? Now we should append the collected and sorted data into our dictionary.

In [14]:
game_info = {
    'Name': name,
    'Release_Date': date,
    'Genre': genre,
    'Price(USD)': price,
    'Score': score,
    'Rating': rating,
    'Votes': votes
}

### 8. Convert the dictionary into the dataframe for further analysis
For the convenience of our analysis afterwards, we need to convert the dictionary into the dataframe.

In [15]:
try:
    df = pd.DataFrame(game_info, columns=['Name', 'Release_Date', 'Genre', 'Price(USD)', 'Score', 'Rating', 'Votes'])
    df.index += 1
    print("Congratulations! The dataframe has been successfully created！")
except:
    print("ERROR! The process has been interrupted, please check your code!")

Congratulations! The dataframe has been successfully created！


### 9. Have a glimpse of the dataframe
We can have a glance at the dataframe by using .head() method in pandas module; or we can use set_option('display.max_rows', None) to browse all the rows.

In [16]:
df.head(5)

,Name,Release_Date,Genre,Price(USD),Score,Rating,Votes
1,Terraria,May-2011,Open World Survival Craft,9.99,8.82,0.98,716854
2,Portal 2,Apr-2011,Platformer,9.99,8.81,0.99,242000
3,The Witcher® 3: Wild Hunt,May-2015,Open World,7.99,8.80,0.98,492729
4,Hades,Sep-2020,Action Roguelike,24.99,8.78,0.99,136441
5,Factorio,Aug-2020,Automation,30.00,8.77,0.99,113682


In [17]:
pd.set_option('display.max_rows', None)
df

,Name,Release_Date,Genre,Price(USD),Score,Rating,Votes
1,Terraria,May-2011,Open World Survival Craft,9.99,8.82,0.98,716854
2,Portal 2,Apr-2011,Platformer,9.99,8.81,0.99,242000
3,The Witcher® 3: Wild Hunt,May-2015,Open World,7.99,8.80,0.98,492729
4,Hades,Sep-2020,Action Roguelike,24.99,8.78,0.99,136441
5,Factorio,Aug-2020,Automation,30.00,8.77,0.99,113682
6,Stardew Valley,Feb-2016,Farming Sim,14.99,8.76,0.98,343741
7,Euro Truck Simulator 2,Oct-2012,Simulation,19.99,8.75,0.98,410858
8,Left 4 Dead 2,Nov-2009,Zombies,9.99,8.74,0.97,476761
9,The Binding of Isaac: Rebirth,Nov-2014,Action Roguelike,7.49,8.72,0.98,133893
10,RimWorld,Oct-2018,Colony Sim,34.99,8.72,0.98,99193


### 10. Find null values in our dataset
Before we start the formal analysis, let's find and filter the null values in our dataset in case of unexpected situation otherwises.

In [18]:
df.isnull().value_counts()

Name   Release_Date  Genre  Price(USD)  Score  Rating  Votes
False  False         False  False       False  False   False    248
       True          False  False       False  False   False      2
dtype: int64

We find that, in the column "Release_Date", there're two null values; except this column, we don't find any other column which include null values.

Therefore, let's find out the specific records of these two null values.

In [19]:
df[df['Release_Date'].isnull() == True]

,Name,Release_Date,Genre,Price(USD),Score,Rating,Votes
44,Don't Starve,None,Survival,9.99,8.59,0.97,87072
117,Mark of the Ninja,None,Stealth,19.99,8.46,0.97,15609


And here they are: Don't Starve and Mark of the Ninja. Since we find that except the release date, all the other records in these two games are filled with correct information. Therefore, we search for the release dates for these two games on Google:

   1. Don't Starve: Apr 2013
   2. Mark of the Ninja: Sep 2012

In [20]:
df.loc[44, 'Release_Date'] = 'Apr-2013'
df.loc[117, 'Release_Date'] = 'Sep-2012'

Now let's check whether our correction is effective:

In [21]:
df.loc[44, 'Release_Date']

'Apr-2013'

In [22]:
df.loc[44, 'Release_Date']

'Apr-2013'

In [23]:
df.isnull().value_counts()

Name   Release_Date  Genre  Price(USD)  Score  Rating  Votes
False  False         False  False       False  False   False    250
dtype: int64

Then we find that in row 21 (Totally Accurate Battle Simulator), the record of Release_Date is "3-days"; Then we need to convert that into the correct format that we want.

In [24]:
df.loc[21]

Name            Totally Accurate Battle Simulator
Release_Date                               3-days
Genre                                       Funny
Price(USD)                                  19.99
Score                                        8.65
Rating                                       0.98
Votes                                       57694
Name: 21, dtype: object

In my current time (4/4/2021), it should be 4/1/2021... Fool's Day for this game's release, what a genius the publisher is!

In [25]:
df.loc[21, 'Release_Date'] = 'Apr-2021'
df.loc[21, 'Release_Date']

'Apr-2021'

### 11. Re-format our data 
Although we get the required data for our exploratory data analysis, we haven't confirm that our data are correctly formatted. Therefore, we need to check the format of our data grouped by the columns and change them into the appropriate formats.

In [26]:
df.dtypes

Name            object
Release_Date    object
Genre           object
Price(USD)      object
Score           object
Rating          object
Votes           object
dtype: object

Object... That's not something that we want in our analysis. Next, we need to change their formats! 

   1. Name: String
   2. Release_Date: Datetime
   3. Genre: String
   4. Price(USD): Float
   5. Score: Float
   6. Rating: Float
   7. Votes: Integer

But first, we copy our original dataframe as our backup, just in case.

In [27]:
df_original = df.copy()

Now let's change the formats!

In [28]:
df['Name'] = df['Name'].astype('str')
df['Release_Date'] =  pd.to_datetime(df['Release_Date'], errors = 'coerce', format = '%b-%Y').dt.to_period('M')
df['Genre'] = df['Genre'].astype('str')
df['Price(USD)'] = df['Price(USD)'].astype('float')
df['Score'] = df['Score'].astype('float')
df['Rating'] = df['Rating'].astype('float')
df['Votes'] = df['Votes'].astype('int')

Now let's look at the data types again:

In [29]:
df.dtypes

Name               object
Release_Date    period[M]
Genre              object
Price(USD)        float64
Score             float64
Rating            float64
Votes               int32
dtype: object

In [30]:
pd.set_option('display.max_rows', None)
df

,Name,Release_Date,Genre,Price(USD),Score,Rating,Votes
1,Terraria,2011-05,Open World Survival Craft,9.99,8.82,0.98,716854
2,Portal 2,2011-04,Platformer,9.99,8.81,0.99,242000
3,The Witcher® 3: Wild Hunt,2015-05,Open World,7.99,8.80,0.98,492729
4,Hades,2020-09,Action Roguelike,24.99,8.78,0.99,136441
5,Factorio,2020-08,Automation,30.00,8.77,0.99,113682
6,Stardew Valley,2016-02,Farming Sim,14.99,8.76,0.98,343741
7,Euro Truck Simulator 2,2012-10,Simulation,19.99,8.75,0.98,410858
8,Left 4 Dead 2,2009-11,Zombies,9.99,8.74,0.97,476761
9,The Binding of Isaac: Rebirth,2014-11,Action Roguelike,7.49,8.72,0.98,133893
10,RimWorld,2018-10,Colony Sim,34.99,8.72,0.98,99193
